## Carga de datos

In [ ]:
%%capture
!pip install mlflow

In [ ]:
import mlflow
import os

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
%%capture
!pip install kagglehub[pandas-dataset]
!pip install optuna
!pip install keras
!pip install pyngrok

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import itertools
import kagglehub
from kagglehub import KaggleDatasetAdapter
import optuna
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import keras
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import os
from pyngrok import ngrok


from sklearn.model_selection import train_test_split

In [ ]:
SEED = 7
token = "2zbl0dEaBIwHYbdy8mg0wAmtd7z_3LPkpSEY88zSz53BdUUbg" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok.connect(5000, "http")
mlflow.set_tracking_uri("http://localhost:5000")
exp_id = mlflow.create_experiment(name="Prediction_addicted", artifact_location="mlruns/")

In [ ]:
file_path = "Students Social Media Addiction.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "adilshamim8/social-media-addiction-vs-relationships",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("Columns:", df.columns)

/tmp/ipython-input-10-1368852353.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Columns: Index(['Student_ID', 'Age', 'Gender', 'Academic_Level', 'Country',
       'Avg_Daily_Usage_Hours', 'Most_Used_Platform',
       'Affects_Academic_Performance', 'Sleep_Hours_Per_Night',
       'Mental_Health_Score', 'Relationship_Status',
       'Conflicts_Over_Social_Media', 'Addicted_Score'],
      dtype='object')


## Preprocesamiento

In [ ]:
df = df.drop(columns=['Student_ID']).drop_duplicates()
X=df.drop(columns=['Addicted_Score'])
y=df['Addicted_Score']

In [ ]:
X['Academic_Level'].unique()
y.unique()

array([8, 3, 9, 4, 7, 2, 5, 6])

In [ ]:
#Mapping academic levels
X["Academic_Level"] = X["Academic_Level"].replace({
    "Undergraduate": 2,
    "Graduate": 3,
    "High School": 1
})

/tmp/ipython-input-13-3165889170.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X["Academic_Level"] = X["Academic_Level"].replace({


In [ ]:
X_cat = X.select_dtypes(include=['object'])
X_num = X.select_dtypes(exclude=['object'])

In [ ]:
X_cat_encoded = pd.get_dummies(X_cat, drop_first=True)
X_cat_encoded

,Gender_Male,Country_Albania,Country_Andorra,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,Country_Bahamas,Country_Bahrain,...,Most_Used_Platform_Snapchat,Most_Used_Platform_TikTok,Most_Used_Platform_Twitter,Most_Used_Platform_VKontakte,Most_Used_Platform_WeChat,Most_Used_Platform_WhatsApp,Most_Used_Platform_YouTube,Affects_Academic_Performance_Yes,Relationship_Status_In Relationship,Relationship_Status_Single
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
701,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
702,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
703,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [ ]:
X_final = pd.concat([X_num, X_cat_encoded], axis=1)
X_final

,Age,Academic_Level,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Conflicts_Over_Social_Media,Gender_Male,Country_Albania,Country_Andorra,Country_Argentina,...,Most_Used_Platform_Snapchat,Most_Used_Platform_TikTok,Most_Used_Platform_Twitter,Most_Used_Platform_VKontakte,Most_Used_Platform_WeChat,Most_Used_Platform_WhatsApp,Most_Used_Platform_YouTube,Affects_Academic_Performance_Yes,Relationship_Status_In Relationship,Relationship_Status_Single
0,19,2,5.2,6.5,6,3,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,22,3,2.1,7.5,8,0,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,20,2,6.0,5.0,5,4,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,18,1,3.0,7.0,7,1,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,21,3,4.5,6.0,6,2,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,20,2,4.7,7.2,7,2,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
701,23,3,6.8,5.9,4,5,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
702,21,2,5.6,6.7,6,3,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
703,24,3,4.3,7.5,8,2,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [ ]:
x_test , x_train , y_test , y_train  = train_test_split(X_final , y, test_size=0.7, random_state=7)
x_val , x_test , y_val , y_test  = train_test_split( x_test , y_test , test_size=0.5, random_state=7)

In [ ]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape , x_val.shape , y_val.shape

((493, 130), (493,), (106, 130), (106,), (105, 130), (105,))

## Bosque de decisión

In [ ]:
def objective_RF(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 5, 15)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 5)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 2)
    criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
    model = RandomForestClassifier(
          n_estimators=n_estimators
          ,max_depth=max_depth
          ,min_samples_split=min_samples_split
          ,min_samples_leaf=min_samples_leaf
          ,criterion=criterion
          ,random_state=SEED
          ).fit(x_train, y_train)
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    return score

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///hp.db",
    study_name="RandomForest",
    load_if_exists=True,
    )

study.optimize(func=objective_RF, n_trials=100, n_jobs=-1)

[I 2025-07-24 02:01:07,155] A new study created in RDB with name: RandomForest
[I 2025-07-24 02:01:07,515] Trial 1 finished with value: 0.8962264150943396 and parameters: {'n_estimators': 14, 'max_depth': 7, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 1 with value: 0.8962264150943396.
[I 2025-07-24 02:01:07,845] Trial 2 finished with value: 0.9528301886792453 and parameters: {'n_estimators': 33, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 2 with value: 0.9528301886792453.
[I 2025-07-24 02:01:08,028] Trial 0 finished with value: 0.839622641509434 and parameters: {'n_estimators': 178, 'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 2 with value: 0.9528301886792453.
[I 2025-07-24 02:01:08,538] Trial 3 finished with value: 0.9150943396226415 and parameters: {'n_estimators': 144, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 2

In [ ]:
params = study.best_params
print(params)

{'n_estimators': 26, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}


In [ ]:
with mlflow.start_run(
        run_name="RF_Optuna", experiment_id=exp_id
        ) as run:

    model = RandomForestClassifier(**params).fit(x_train, y_train)
    acc = accuracy_score(y_val, model.predict(x_val))
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="Prediction_addicted",
        input_example = x_train.sample(1)
    )
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy": acc
        })

2025/07/24 02:01:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer col

🏃 View run RF_Optuna at: http://localhost:5000/#/experiments/1/runs/ed3f6da9537849d9a4a765784d50b080
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
best_model  = mlflow.sklearn.load_model(f"models:/Prediction_addicted/latest")
report = classification_report(y_val, best_model.predict(x_val))
print(report)

              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         9
           5       0.94      1.00      0.97        16
           6       1.00      0.92      0.96        13
           7       1.00      1.00      1.00        32
           8       0.97      1.00      0.98        28
           9       1.00      0.83      0.91         6

    accuracy                           0.98       105
   macro avg       0.99      0.97      0.97       105
weighted avg       0.98      0.98      0.98       105



In [ ]:
params = {'n_estimators':range(50,101,10), 'criterion':['gini','entropy'] }
forest = RandomForestClassifier()
grid = GridSearchCV(forest, params, cv=5, verbose=2)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.2s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.2s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.2s
[CV] END ....................criterion=gini, n_estimators=70; total time=   0.3s
[CV] END ....................criterion=gini, n_e

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': range(50, 101, 10)},
             verbose=2)

In [ ]:
grid.best_params_

{'criterion': 'entropy', 'n_estimators': 50}

In [ ]:

print(classification_report(y_val, grid.predict(x_val)))

              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         9
           5       1.00      1.00      1.00        16
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00        32
           8       0.97      1.00      0.98        28
           9       1.00      0.83      0.91         6

    accuracy                           0.99       105
   macro avg       1.00      0.98      0.98       105
weighted avg       0.99      0.99      0.99       105



Veamos si podemos mejorarlo un poco más

In [ ]:
new_params = {'n_estimators': range(40,60,1), 'criterion': ['entropy']}
forest_2 = RandomForestClassifier()
grid_2 = GridSearchCV(forest_2, new_params, cv=5, verbose=2)
grid_2.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=42; total time=   0.1s
[CV] END .................criterion=entropy, n_

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['entropy'],
                         'n_estimators': range(40, 60)},
             verbose=2)

In [ ]:
grid_2.best_params_

{'criterion': 'entropy', 'n_estimators': 52}

In [ ]:
with mlflow.start_run(
        run_name="RF_GS", experiment_id=exp_id
        ) as run:

    model = RandomForestClassifier(**grid_2.best_params_).fit(x_train, y_train)
    acc = accuracy_score(y_val, model.predict(x_val))
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="Prediction_addicted_2",
        input_example = x_train.sample(1)
    )
    mlflow.log_params(grid_2.best_params_)
    mlflow.log_metrics({
        "accuracy": acc
        })

2025/07/24 02:02:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer col

🏃 View run RF_GS at: http://localhost:5000/#/experiments/1/runs/f9b366295e3e4592ab2d9ac3e564db04
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
best_model  = mlflow.sklearn.load_model(f"models:/Prediction_addicted_2/latest")
report = classification_report(y_val, best_model.predict(x_val))
print(report)

              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         9
           5       1.00      1.00      1.00        16
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00        32
           8       0.97      1.00      0.98        28
           9       1.00      0.83      0.91         6

    accuracy                           0.99       105
   macro avg       1.00      0.98      0.98       105
weighted avg       0.99      0.99      0.99       105



## Red neuronal

Finalmente, intentamos con una red neuronal sencilla

In [ ]:
y_train_encoded = pd.get_dummies(y_train, drop_first=False)
y_val_encoded = pd.get_dummies(y_val, drop_first=False)
y_test_encoded = pd.get_dummies(y_test, drop_first=False)

print("Shape of y_train after one-hot encoding:", y_train_encoded.shape)
print("Shape of y_val after one-hot encoding:", y_val_encoded.shape)
print("Shape of y_test after one-hot encoding:", y_test_encoded.shape)

Shape of y_train after one-hot encoding: (493, 7)
Shape of y_val after one-hot encoding: (105, 7)
Shape of y_test after one-hot encoding: (106, 8)


Para asegurar la compatibilidad del experimento y considerando que hay muy pocos registros de 'addicted score' 2 (tan pocos que todos quedaron en el split test), se eliminarán esos registros.

In [ ]:
NN = keras.models.Sequential()
NN.add(keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
NN.add(keras.layers.Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['precision', 'recall'])

In [ ]:
%%capture
NN.fit(x_train, y_train_encoded, epochs=100, validation_data=(x_val, y_val_encoded))

In [ ]:
y_test_encoded.drop(2, axis=1, inplace=True)
y_test_encoded.shape

(106, 7)

In [ ]:
NN.evaluate(x_test, y_test_encoded)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2195 - precision: 0.9528 - recall: 0.9583


[0.2618859112262726, 0.9339622855186462, 0.9428571462631226]

Como se puede ver, la red neuronal se desempeña bien, pero el bosque de decisión le gana por tres puntos porcentuales.

## Despliegue

In [ ]:
!mkdir src

In [ ]:
%%writefile ./src/mlcli.py
from argparse import ArgumentParser
import mlflow
import pandas
mlflow.set_tracking_uri("http://localhost:5000")

def main():
    parser = ArgumentParser(
            prog="Social Media Addiction Classifier",
            description="CLI para modelo de prediccion de adiccion a redes sociales"
            )
    parser.add_argument("-p", "--filepath", type=str, required=True, help="Ruta al archivo csv que contiene la serie de python")
    args = parser.parse_args()

    if args.filepath:
      try:
        data = pandas.read_csv(args.filepath, header=0).iloc[:,1:]
      except Exception as e:
        print(e)

    model = mlflow.pyfunc.load_model("models:/Prediction_addicted/latest")
    prediction = model.predict(data)
    print(f"El puntaje predicho de adicción es: {prediction}")

if __name__ == "__main__":
    main()

Overwriting ./src/mlcli.py


In [ ]:
x_test.iloc[[3, 7, 10]].to_csv("test.csv")

In [ ]:
!python ./src/mlcli.py --help

usage: Social Media Addiction Classifier [-h] -p FILEPATH

CLI para modelo de prediccion de adiccion a redes sociales

options:
  -h, --help            show this help message and exit
  -p FILEPATH, --filepath FILEPATH
                        Ruta al archivo csv que contiene la serie de python


In [ ]:
!python ./src/mlcli.py -p ./test.csv

El puntaje predicho de adicción es: [6 7 4]
